In [1]:
## timelimit = 02:45:00 
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["data", "estimation"]:
    path[folder] = f"{path_file}/{folder}"

job_id = 0 #int(os.environ["SLURM_ARRAY_TASK_ID"])

In [2]:
from numpy.random import RandomState

In [3]:
from regime.jump import *
from regime.simulation_helper import *

In [4]:
n_c = 3
batch_size = 4

In [5]:
random_state = RandomState(10)
model_discrete = jump_model(n_c, state_type="discrete", random_state=random_state)
model_cont_mode = jump_model(n_c, state_type="cont", grid_size=.05, mode_loss=True, random_state=random_state)
model_cont = jump_model(n_c, state_type="cont", grid_size=.05, mode_loss=False, random_state=random_state)
model_dict_jump = {"discrete": model_discrete, "cont-mode": model_cont_mode, "cont": model_cont}

In [6]:
param_grid = generate_param_grid()

[1.00000000e-02 1.00000000e-01 1.00000000e+00 1.00000000e+01
 2.15443469e+01 4.64158883e+01 1.00000000e+02 2.15443469e+02
 4.64158883e+02 1.00000000e+03 2.15443469e+03 4.64158883e+03
 1.00000000e+04 1.00000000e+05 1.00000000e+06]


In [7]:
key_feat_jump = ["zhengB"]

In [9]:
job_id = 3

In [8]:
key_data = "3-state-daily"
name = "modelParams"
key_len = 250
key_feat = "zhengB"
key_model = "discrete"
folder = "estimation"

In [10]:
start = job_id * batch_size; end = start + batch_size

In [13]:
Zs = load_file(path, "data", key_data, "Zs", key_len)[start:end]

In [19]:
filename = "Xs_250_zhengB.npy"

In [20]:
Xs = np.load(f"{path['data']}/{key_data}/{filename}")[start:end]

In [28]:
model_params_arr, labels_arr, proba_arr = model_fit_batch(model_discrete, Xs, Zs, True)#model_fit_batch_with_params(model_discrete, Xs, Zs, param_grid, True)

In [ ]:
def model_fit_batch(model, Xs, Zs, align=True):
    """
    fit a model on a batch of data. save model parameters, and fitted labels & proba.
    need the true labels to do alignments (i.e. the permutation with the highest overall accuracy)

        
    Paramaters:
    ---------------------
    model: a model instance.
    
    Xs: array of size (n_t, n_s, n_f)
        input data
        
    Zs: array (n_t, n_s)
        true labels
        
    Returns:
    -------------------------
    model_params_arr: (n_t, n_c**2 + n_c)
    
    labels_arr: array of size (n_t, n_s)
    
    proba_arr: array of size (n_t, n_s, n_c)
    """
    n_t, n_s, _ = Xs.shape
    n_c = model.n_components

    res_list = []
    # estimate
    # for i_t in trange(n_t):
    for i_t in range(n_t):
        X, Z = Xs[i_t], Zs[i_t]
        # fit
        model.fit(X)
        # save dict result
        res_list.append(extract_results_from_model(model, X_=X[:, 0]))
    # dict of results
    res = combine_list_dict(res_list)
    # align with true labels
    if align:
        res = align_estimation_results_batch(Zs, res)
    # combine model params
    model_params_arr = combine_model_param_estimation(res["means_"], res["covars_"], res["transmat_"])
    labels_arr = res["labels_"]; proba_arr = res["proba_"]
    return model_params_arr, labels_arr, proba_arr

In [67]:
model_params_arr, labels_arr, proba_arr = model_fit_batch(model, Xs, Zs, align=True)

In [70]:
proba_arr.shape

(4, 250, 3)

In [46]:
model=model_discrete

In [47]:
n_t, n_s, _ = Xs.shape
n_c = model.n_components

res_list = []

In [48]:
i_t=0

In [49]:
X, Z = Xs[i_t], Zs[i_t]
# fit
model.fit(X)
# save dict result
res_list.append(extract_results_from_model(model, X_=X[:, 0]))

In [55]:
res = combine_list_dict(res_list)

In [ ]:
n_t, n_s, _ = Xs.shape
n_c = model.n_components

res_list = []
# estimate
# for i_t in trange(n_t):
for i_t in range(n_t):
    X, Z = Xs[i_t], Zs[i_t]
    # fit
    model.fit(X)
    # save dict result
    res_list.append(extract_results_from_model(model, X_=X[:, 0]))
# dict of results
res = combine_list_dict(res_list)
# align with true labels
if align:
    res = align_estimation_results_batch(Zs, res)
# combine model params
model_params_arr = combine_model_param_estimation(res["means_"], res["covars_"], res["transmat_"])
labels_arr = res["labels_"]; proba_arr = res["proba_"]

In [29]:
model_params_arr.shape

(4, 6)

In [26]:
proba_arr.shape

(4, 250, 2, 15)

In [ ]:
def model_fit_many_datas_models(key_data_list, key_feat_list, model_dict, param_grid, path, job_id, batch_size, n_s_list=None, align=True):
    """
    train a collection of models, w/ hyperparams to tune, on a batch of data from many datasets.
    can specify the seq length to fit, or fit all in the folder.
    """
    key_data_list, key_feat_list = raise_str_to_list(key_data_list), raise_str_to_list(key_feat_list)
    #
    start = job_id * batch_size; end = start + batch_size
    N_combos = len(key_data_list) * len(key_feat_list) * len(model_dict)
    count = 0; time_old = time.time(); total_time=0.
    for key_data, key_feat in product(key_data_list, key_feat_list):
        folder = f"{path['data']}/{key_data}"
        if n_s_list is not None:
            filenames = [f"Xs_{n_s}_{key_feat}.npy" for n_s in n_s_list]
        else:
            filenames = filter_filenames_in_folder(folder, key_feat)
        for key_model, model in model_dict.items():
            count+=1; print(f"{count}/{N_combos} combo starts.")
            for filename in filenames:
                Xs = np.load(f"{folder}/{filename}")[start:end]
                n_s = int(filename.split('_')[1]); Zs = load_file(path, "data", key_data, "Zs", n_s)[start:end]
                # train the model, on a param grid, on a batch of data
                model_params_arr, labels_arr, proba_arr = model_fit_batch_with_params(model, Xs, Zs, param_grid, align)
                # save results
                save_file({"modelParams": model_params_arr,
                          "labels": labels_arr,
                          "proba": proba_arr}, 
                          path, "estimation", key_data, None, n_s, key_feat, key_model, job_id)
            time_now = time.time(); time_this_iter = time_now-time_old; total_time += time_this_iter; time_old = time_now
            print(f"{count}/{N_combos} combo done. Time of this combo: {print_seconds(time_this_iter)}s. Total time: {print_seconds(total_time)}s.")
    return 
